<a href="https://colab.research.google.com/github/anshulp2912/PhotoEditor-OpenCV/blob/master/source/PhotoEditor_opencv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center><font color='red'>PHOTO EDITOR USING OPEN-CV COMMANDS</font></center>
#### <center>Bring creativity to your photos</center>
#### <center>Apply Instagram-like filters to your photos</center>

<center>Checkout: github.com/anshulp2912/PhotoEditor-OpenCV</center>


### Import Libraries

In [1]:
!pip install gradio
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from PIL import Image 
import gradio as gr

## Define Functions for each Control and Effects

### Brightness Control

In [2]:
def brightness_control(image_path,brightness):
  brightness = int(((brightness + 127)*(127 + 127)/(100 + 100)) - 127)
  image = cv2.imread(image_path)
  if brightness > 0:
      shadow = brightness
      highlight = 255
  else:
      shadow = 0
      highlight = 255 + brightness
  alpha_b = (highlight - shadow)/255
  gamma_b = shadow
  
  image = cv2.addWeighted(image, alpha_b, image, 0, gamma_b)
  #cv2_imshow(image)
  cv2.imwrite('temp.jpg',image)

### Contrast Control

In [3]:
def contrast_control(image_path, contrast):
  contrast = int(((contrast + 64)*(64 + 64)/(100 + 100)) - 64)
  image = cv2.imread(image_path)
  if contrast != 0:
    f = 131*(contrast + 127)/(127*(131-contrast))
    alpha_c = f
    gamma_c = 127*(1-f)
    
    image = cv2.addWeighted(image, alpha_c, image, 0, gamma_c)
  #cv2_imshow(image)
  cv2.imwrite('temp.jpg',image)

### Saturation Control

In [4]:
def saturation_control(image_path,value):
  value = 1 + (value/100)
  image = cv2.imread(image_path)
  hsvImg = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
  hsvImg[...,1] = hsvImg[...,1]*value
  image=cv2.cvtColor(hsvImg,cv2.COLOR_HSV2BGR)
  #cv2_imshow(image)
  cv2.imwrite('temp.jpg',image)

### Hue Control

In [5]:
def hue_control(image_path,value):
  image = cv2.imread(image_path)
  hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  h, s, v = cv2.split(hsv)
  h += value
  final_hsv = cv2.merge((h, s, v))
  image = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
  #cv2_imshow(image)
  cv2.imwrite('temp.jpg',image)

### Vignette Control

In [6]:
def vignette_control(image_path,vignette):
  vignette = int(175.00/((vignette)/(100.00)))
  image = cv2.imread(image_path)
  rows,cols = image.shape[:2]
  zeros = np.copy(image)
  zeros[:,:,:] = 0
  a = cv2.getGaussianKernel(cols,vignette)
  b = cv2.getGaussianKernel(rows,vignette)
  c = b*a.T
  d = c/c.max()
  zeros[:,:,0] = image[:,:,0]*d
  zeros[:,:,1] = image[:,:,1]*d
  zeros[:,:,2] = image[:,:,2]*d
  #cv2_imshow(zeros)
  cv2.imwrite('temp.jpg',zeros)

### Sharpen Control

In [7]:
def sharpen_control(image_path, amount):
  amount = int(((amount - 1)*(3 - 1)/(100 - 0)) + 1)
  image = cv2.imread(image_path)
  kernel_size=(5, 5)
  sigma=1.0 
  threshold=0
  blurred = cv2.GaussianBlur(image, kernel_size, sigma)
  sharpened = float(amount + 1) * image - float(amount) * blurred
  sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
  sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
  sharpened = sharpened.round().astype(np.uint8)
  if threshold > 0:
      low_contrast_mask = np.absolute(image - blurred) < threshold
      np.copyto(sharpened, image, where=low_contrast_mask)
  #cv2_imshow(sharpened)
  cv2.imwrite('temp.jpg',sharpened)

### Cartoon Effect

In [8]:
def cartoon_effect(image_path):
  img_rgb = cv2.imread(image_path)
  num_down = 2
  num_bilateral = 7
  img_color = img_rgb
  for _ in range(num_down):
    img_color = cv2.pyrDown(img_color)
  for _ in range(num_bilateral):
    img_color = cv2.bilateralFilter(img_color, d=9, sigmaColor=9, sigmaSpace=7)
  for _ in range(num_down):
    img_color = cv2.pyrUp(img_color)
  img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
  img_blur = cv2.medianBlur(img_gray, 7)

  img_edge = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,
                                  blockSize=9, C=2)
  img_edge = cv2.cvtColor(img_edge, cv2.COLOR_GRAY2RGB)
  img_cartoon = cv2.bitwise_and(img_color, img_edge)
  #cv2_imshow(img_cartoon)
  cv2.imwrite('temp.jpg',img_cartoon)

### Blur Effect

In [9]:
def blur_filter(image_path):
  image = cv2.imread(image_path)
  image = cv2.GaussianBlur(image,(5,5),cv2.BORDER_DEFAULT)
  #cv2_imshow(image)
  cv2.imwrite('temp.jpg',image)

### Edge Effect

In [10]:
def edge_filter(image_path):
  image = cv2.imread(image_path)
  image = cv2.Canny(image,100,300)
  #cv2_imshow(image)
  cv2.imwrite('temp.jpg',image)

### Vintage Effect

In [11]:
def vintage_filter(image_path):
  image = cv2.imread(image_path)
  rows, cols = image.shape[:2]
  # Create a Gaussian filter
  kernel_x = cv2.getGaussianKernel(cols,200)
  kernel_y = cv2.getGaussianKernel(rows,200)
  kernel = kernel_y * kernel_x.T
  filter = 255 * kernel / np.linalg.norm(kernel)
  vintage_im = np.copy(image)
  # for each channel in the input image, we will apply the above filter
  for i in range(3):
      vintage_im[:,:,i] = vintage_im[:,:,i] * filter
  #cv2_imshow(vintage_im)
  cv2.imwrite('temp.jpg',vintage_im)

### Black & White Effect

In [12]:
def blackwhite_filter(image_path):
  image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
  #cv2_imshow(image)
  cv2.imwrite('temp.jpg',image)

### MonoChrome Effect

In [13]:
def monochrome_filter(image_path):
  image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
  (thresh, im_bw) = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
  #cv2_imshow(im_bw)
  cv2.imwrite('temp.jpg',im_bw)

### ADJUSTMENT AVAILABLE:
- Brightness
- Contrast
- Saturation
- Hue
- Vignette
- Sharpen<br>

### EFFECTS AVAILABLE:
- Cartoon
- Edge
- Vintage
- Blur
- Black & White
- Monochrome


## PHOTO EDITOR MAIN FUNCTION

In [14]:
def photoEditor(imageInput, brightnessValue, contrastValue, saturationValue, hueValue, vignetteValue, sharpenValue, effectList):
  image_inp = Image.fromarray(imageInput)
  image_inp.save("temp.jpg") 

  image = cv2.imread("temp.jpg")
  image = cv2.resize(image,(800,800))
  cv2.imwrite('temp.jpg', image)

  if brightnessValue != 0.00:
    brightness_control('temp.jpg',brightnessValue)

  if contrastValue != 0.00:
    contrast_control('temp.jpg',contrastValue)

  if saturationValue != 0.00:
    saturation_control('temp.jpg',saturationValue)

  if hueValue != 0.00:
    hue_control('temp.jpg',hueValue)

  if vignetteValue != 0.00:
    vignette_control('temp.jpg',vignetteValue)
    
  if sharpenValue != 0.00:
    sharpen_control('temp.jpg',sharpenValue)

  if len(effectList) != 0:
    if 'Cartoon' in effectList:
      cartoon_effect('temp.jpg')

    if 'Edge' in effectList:
      edge_filter('temp.jpg')

    if 'Vintage' in effectList:
      vintage_filter('temp.jpg')

    if 'Blur' in effectList:
      blur_filter('temp.jpg')

    if 'Black & White' in effectList:
      blackwhite_filter('temp.jpg')

    if 'Monochrome' in effectList:
      monochrome_filter('temp.jpg')

  im = Image.open('temp.jpg')
  return im

### GRADIO PARAMETERS

In [15]:
imageInput = gr.inputs.Image(label='Upload an Image')
brightnessSlider = gr.inputs.Slider(minimum=-100, maximum=100, default=0, label='Brightness')
contrastSlider = gr.inputs.Slider(minimum=-100, maximum=100, default=0, label='Contrast')
saturationSlider = gr.inputs.Slider(minimum=0, maximum=100, default=0, label='Saturation')
hueSlider = gr.inputs.Slider(minimum=0, maximum=100, default=0, label='Hue')
vignetteSlider = gr.inputs.Slider(minimum=0, maximum=100, default=0, label='Vignette')
sharpenSlider = gr.inputs.Slider(minimum=0, maximum=100, default=0, label='Sharpen')
effectCheckboxes = gr.inputs.CheckboxGroup(["Cartoon", "Edge", "Vintage", "Blur", "Black & White", "Monochrome"], label='Effects')
imageOutput = gr.outputs.Image(label='Edited Image')

### Launch PhotoEditor Interface

In [16]:
gr.Interface(fn=photoEditor, 
             inputs=[imageInput, brightnessSlider, contrastSlider,
                     saturationSlider, hueSlider, vignetteSlider, sharpenSlider, effectCheckboxes],
            outputs=imageOutput, title='OpenCV PhotoEditor', description='Apply adjustments to your photo and make it go Glam!!. For more info, checkout github.com/anshulp2912/PhotoEditor-OpenCV ' ,allow_flagging=False, live=True).launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on External URL: https://27947.gradio.app
Interface loading below...


(<gradio.networking.serve_files_in_background.<locals>.HTTPServer at 0x7fbfaacf7eb8>,
 'http://127.0.0.1:7860/',
 'https://27947.gradio.app')